# ETL Project
##### Adam Cook
##### ac7nm

### First, import all necessary packages, and change the path to your desired working directory

In [9]:
import os
import pandas as pd
import mysql.connector
import pymysql
import sqlite3
import numpy as np
import json
os.chdir('/Users/adam/Desktop/Data Science Systems')

### Retrieve the dataset from Kaggle and save it in your working directory: https://www.kaggle.com/datasets/mathchi/hitters-baseball-data

### Once you have done this, run the code below that will create the function necessary to make the desired changes to the data

In [10]:
def etl(csv_name, desired_format, desired_name):
    try:
        df = pd.read_csv(csv_name) #Reading in the csv from local working directory
    except:
        #if an error occurs here, the user is told that the csv could not be read in
        return 'File does not exist in working directory. Remember to wrap the name in quotes and include the .csv extension'
    try:
        df['Average'] = (df['Hits']/df['AtBat']) #making column changes to the dataframe - got rid of the career columns and added a batting average column
        df1 = df.drop(df.columns[[7,8,9,10,11,12]],axis = 1)
    except:
        #If the column edits return an error, the user is informed that the dataframe could not be edited 
        return 'Unable to make changes to dataframe'
    if desired_format == 'csv':
        try:
            data = df1 #If the user wants a csv, no changes have to be made 
            data.to_csv(desired_name)#This line takes the csv and saves it to the local directory
            shape = data.shape #setting a variable for the number of rows and columns
            #returns successful message and the shape of the dataframe
            return f"Modified csv file written to local directory with size: {shape}", data.head()
        except:
            #If an error occurs here, the user is informed that the new csv was not saved to their machine
            return 'Modified csv file could not be written to local directory'
    elif desired_format == 'json':
        try:
            data = df1.to_json(desired_name)#If the user wants json, this converts the csv to a json object and writes it to the directory
            o = open(desired_name)
            data1 = json.load(o)#These two lines make it so the json can be displayed in the notebook
            size = len(data1)
            #returns a success message, the size of the json object, and the json object 
            return f"json file saved to local directory with size: {size}", data1
        except:
            #if an error occurs here, the user is informed the file was not converted
            return 'file could not be converted to json'
    elif desired_format == 'db':
        try:
            con = sqlite3.connect("Hitters.db") #creating a connection to a database in sqlite
            cur = con.cursor() #creating a cursor
            df1.to_sql(name='Hitters', con=con, if_exists = 'replace') #sending the dataframe to an sqlite table
            cur.execute("""SELECT COUNT(*) FROM pragma_table_info('Hitters');""")
            col_count = cur.fetchall()#These two lines get the number of columns of the database table
            cur.execute(""" SELECT COUNT(*) FROM Hitters;""")
            row_count = cur.fetchall()#These two lines get the number of rows of the database table
            cur.execute(""" SELECT * FROM Hitters;""")
            table = cur.fetchall() #These two lines fetch the table to display
            #A success message is returned with the number of rows and columns in addition to the values in the table 
            return f'SQLite database table created with {row_count} rows and {col_count} columns', table 
        except:
            #if an error occurs, the user is informed the csv was not converted
            return 'csv could not be converted to SQL database table'
    

## User Instructions:
- Within the parantheses below, enter:
    - The name of the csv file in your directory including the file extension
    - The format of your desired output, either:
        - 'csv' for a csv file,
        - 'json' for a json object,
        - 'db' for an SQLite database table
    - Your desired name of the output file to be saved to your directory, including the file extension that matches your desired format
- Enter these items in the format ('name','format','desired name'), ensuring there are commas between the items and all three of them are wrapped in quotes
- Execute the code block

In [ ]:
etl()